# iris 데이터를 사용하여 KNN 알고리즘을 직접 작성해보자

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

iris=load_iris()
X=iris.data #data size 150개 (feature 4개)
y=iris.target # 결과 값 ( 어떤 꽃 종류인지 0 ,1 ,2 중에 있다.)
y_name=iris.target_names

y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

# 유클리디안 거리를 이용한 거리 구하기


In [6]:
import numpy as np

a = np.array([2, 1, 60, 139, 0])
b = np.array([3, 2, 80, 148, 0])

def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))


# 거리와 index k개반환하기

In [75]:
def find_nearest_neighbors(x, iris_data, k): 
    
    distance_iris = np.zeros(len(iris_data))
    # 거리를 data와 비교해서 각각 구한다.
    for i in  range(len(iris_data)):
        distance_iris[i] = euclidean_distance(iris_data[i],x )
    # index (반환 거리가 작은 순으로 정렬- k개를 뽑는다)    
    sorted_index = np.argsort(distance_iris)[:k]  
    
    sorted_distance=np.sort(distance_iris)[:k]
        
    return sorted_index,sorted_distance

In [10]:
sorted_a, sorted_b = find_nearest_neighbors(X[149], X, 5)
print(sorted_a)
print(sorted_b)

[149 127 138 142 101]
[0.         0.28284271 0.31622777 0.33166248 0.33166248]


# majority vote를 이용하기

In [82]:
from collections import Counter

def majority_vote(sorted_index,k,y):
    
    target_result = np.zeros(k)
    
    for i in range(k):
        target_result[i] = y[sorted_index[i]]
    
    num_cnt=Counter(target_result)
    # 제일 많이 있는 것 return 한다.
    max_cnt = -1
    for num in num_cnt:
        if num_cnt[num] > max_cnt:
            max_cnt=num_cnt[num]
            max_result=int(num)

    return max_result

In [65]:
majority_vote(sorted_a,5,y)

2

# weighted vote를 이용하기

In [78]:
def weighted_majority_vote(sorted_index,sorted_distance,k,y):
    iris_sum = np.zeros(3)
    for i in range(k):
        # weight를 거리의 역수로 생각하자
        iris_sum[y[sorted_index[i]]]+=1/sorted_distance[i]
    max_iris=-1
    for i in range(3):
        if iris_sum[i]>max_iris:
            max_iris=iris_sum[i]
            max_result=i
            
    return max_result  
        

In [29]:
sorted_a, sorted_b = find_nearest_neighbors(X[146], X[:145], 5)
weighted_majority_vote(sorted_a,sorted_b,5)


int

# KNN 결과값 확인

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

iris=load_iris()
X=iris.data #data size 150개 (feature 4개)
y=iris.target # 결과 값 ( 어떤 꽃 종류인지 0 ,1 ,2 중에 있다.)
y_name=iris.target_names

In [10]:
num=14
#test data, train data 분리
X_train=[]
X_test=[]
for i in range(len(X)):
    if i==num:
        X_test.append(X[i])
        num+=15
    else:
        X_train.append(X[i])

num=14
y_train=[]
y_test=[]
for i in range(len(y)):
    if i==num:
        y_test.append(y[i])
        num+=15
    else:
        y_train.append(y[i])
        

# KNN class 만들기

In [33]:
import numpy as np

class KNN_class:
    def __init__(self,X_train,y_train):
        self.X_train=X_train
        self.y_train=y_train
        
    def euclidean_distance(self,a, b):
        # 유클리드 거리 방식을 사용하였다.
        return np.sqrt(np.sum((a - b) ** 2))
    
    def find_nearest_neighbors(self, x, k): 
        
        distance_iris = np.zeros(len(self.X_train))
        # 거리를 data와 비교해서 각각 구한다.
        for i in  range(len(self.X_train)):
            distance_iris[i] = self.euclidean_distance(self.X_train[i],x )
        # index (반환 거리가 작은 순으로 정렬- k개를 뽑는다)    
        sorted_index = np.argsort(distance_iris)[:k]  
    
        sorted_distance=np.sort(distance_iris)[:k]
        
        return sorted_index,sorted_distance
    
    def majority_vote(self, sorted_index, k):
        
        target_result = np.zeros(k)
    
        for i in range(k):
            target_result[i] = self.y_train[sorted_index[i]]
        
        iris_sum=np.zeros(3)
        for i in range(k):
            #가까운 것의 개수를 배열에 넣는다. (나올때마다 추가)
            iris_sum[int(target_result[i])]+=1
        
        # 제일 가까이 있는 것의 index를 return 한다.
        max_iris=-1
        for i in range(3):
            if iris_sum[i]>max_iris:
                max_iris=iris_sum[i]
                max_result=i

        return max_result
    
    def weighted_majority_vote(self, sorted_index,sorted_distance,k):
        iris_sum = np.zeros(3)
        for i in range(k):
            # weight를 거리의 역수로 생각하자
            iris_sum[self.y_train[sorted_index[i]]]+=1/sorted_distance[i]
            
        max_iris=-1
        for i in range(3):
            if iris_sum[i]>max_iris:
                max_iris=iris_sum[i]
                max_result=i
            
        return max_result 
        
    

# main.py

In [34]:
import numpy as np
from sklearn.datasets import load_iris

# iris 데이터를 받아온다.
iris=load_iris()
#data size 150개 (feature 4개)
X=iris.data 
# 결과 값
y=iris.target
# 결과 (꽃의 이름)
y_name=iris.target_names

#train data(140개), test data(10개) 분리시키자
num=14
X_train=[]
X_test=[]
for i in range(len(X)):
    if i==num:
        X_test.append(X[i])
        num+=15
    else:
        X_train.append(X[i])

num=14
y_train=[]
y_test=[]
for i in range(len(y)):
    if i==num:
        y_test.append(y[i])
        num+=15
    else:
        y_train.append(y[i])

# KNN 분류기 객체를 만든다.
KNN_classifier = KNN_class(X_train,y_train)

# k가 3일때, 5일때, 10일때 
k_num=[3,5,10]

for k in k_num:
    print("========================== k is",k,"==========================")
    for j in range(2):
        if j ==0:
            print("====================== majority vote ======================")
        else :
            print("=================== weighted majority vote ===================")
        accuracy=0
        for i in range(len(X_test)):
            # 차례로 거리가 가까운 순으로 index 배열, 주어진 데이터와 가까운 것의 거리 배열을 나타낸다.
            sorted_index,sorted_distance=KNN_classifier.find_nearest_neighbors(X_test[i], k)
            if j==0:
                # majority vote
                predict_iris=KNN_classifier.majority_vote(sorted_index,k)         
            else:
                # weighted majority vote 
                predict_iris=KNN_classifier.weighted_majority_vote(sorted_index,sorted_distance,k)          
            # 실제 데이터 값과 같은지 확인한다.
            print("Test Data index: ",i,"Computed class: ",y_name[predict_iris],", True class: ",y_name[y_test[i]])
            # 실제 데이터 값과 같으면 accuracy를 하나 올려준다.
            if y_name[predict_iris] == y_name[y_test[i]]:
                accuracy+=1
        # accuracy를 출력하자        
        if j==0:
            print("majority vote k =",k,", Accuracy :" ,accuracy/len(X_test))
        else:
            print("weighted majority vote k =",k,", Accuracy :" ,accuracy/len(X_test))

========================== k is 3 ==========================
====================== majority vote ======================
Test Data index:  0 Computed class:  setosa , True class:  setosa
Test Data index:  1 Computed class:  setosa , True class:  setosa
Test Data index:  2 Computed class:  setosa , True class:  setosa
Test Data index:  3 Computed class:  versicolor , True class:  versicolor
Test Data index:  4 Computed class:  versicolor , True class:  versicolor
Test Data index:  5 Computed class:  versicolor , True class:  versicolor
Test Data index:  6 Computed class:  virginica , True class:  virginica
Test Data index:  7 Computed class:  versicolor , True class:  virginica
Test Data index:  8 Computed class:  virginica , True class:  virginica
Test Data index:  9 Computed class:  virginica , True class:  virginica
majority vote k = 3 , Accuracy : 0.9
=================== weighted majority vote ===================
Test Data index:  0 Computed class:  setosa , True class:  setosa
Test